In [117]:
import pandas as pd
import sqlite3


2. Album vs. Individual Track Purchases:

- Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
- A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
- Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).

In [148]:
conn = sqlite3.connect('chinook.db')
cursor = conn.cursor()

tables = pd.read_sql_query("select name from sqlite_master WHERE type='table'",conn)
dfs={}
for i in tables['name']:
    dfs[i]=pd.read_sql_query(f'select * from {i}',conn)

In [149]:
track_count_album = dfs['tracks'].groupby('AlbumId')['TrackId'].count().reset_index()

ff = pd.merge(dfs['invoice_lines'],dfs['tracks'],on='TrackId').merge(dfs['invoices'],on='InvoiceId')
track_count_purch = ff[['CustomerId','AlbumId','TrackId']].groupby(['CustomerId','AlbumId']).agg(count_tracks=('TrackId','count')).reset_index()

result = pd.merge(track_count_purch,track_count_album,how='outer',left_on=['AlbumId','count_tracks'], right_on=['AlbumId','TrackId'],indicator=True)

n_track_buy_customers = result[result['_merge'] != 'both']['CustomerId'].nunique()
n_album_buy_customers = result[result['_merge']=='both']['CustomerId'].nunique()

n_customers = dfs['customers']['CustomerId'].nunique()


print(f'Total customers: {n_customers}')
print(f'Customers who prefer to buy indvidual tracs: {n_track_buy_customers}, {int(n_track_buy_customers / n_customers * 100)}%')
print(f'Customers who prefer to buy full albums: {n_album_buy_customers}, {int(n_album_buy_customers / n_customers * 100)}%')

Total customers: 5
Customers who prefer to buy indvidual tracs: 2, 40%
Customers who prefer to buy full albums: 3, 60%
